In [45]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [117]:
#1 задание. Есть пользователи, которые ставят друг другу лайки. Создать таблицы для хранения инфы. 
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                        database="postgres", 
                        user='postgres',
                        password='120303', 
                        host='localhost', 
                        port='5432'
                    )
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [118]:
import psycopg2 as pg_driver

db = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='120303', 
                        host='localhost', 
                        port='5432'
                    )



def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            print(sql_command)
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [119]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='120303', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [120]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (1, to_timestamp('04-08-2019 23:38:00', 'dd-mm-yyyy hh24:mi:ss'), 'Katya'),
                 (2, to_timestamp('25-10-2020 11:20:00', 'dd-mm-yyyy hh24:mi:ss'), 'Julia'),
                 (3, to_timestamp('04-10-2016 08:44:00', 'dd-mm-yyyy hh24:mi:ss'), 'Olya'),
                 (4, to_timestamp('12-03-2018 15:56:00', 'dd-mm-yyyy hh24:mi:ss'), 'Lena'),
                 (5, to_timestamp('01-12-2021 22:07:00', 'dd-mm-yyyy hh24:mi:ss'), 'Dima')
                 
            
        """
execute_query(query)

In [121]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('04-08-2019 23:38:00', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('25-10-2020 11:20:00', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (3, to_timestamp('04-10-2016 08:44:00', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (4, to_timestamp('12-03-2018 15:56:00', 'dd-mm-yyyy hh24:mi:ss'), 2),
                 (5, to_timestamp('01-12-2021 22:07:00', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 
            
        """
execute_query(query)

In [122]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('04-08-2019 23:38:00', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (1, to_timestamp('04-08-2019 23:38:00', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (2, to_timestamp('25-10-2020 11:20:00', 'dd-mm-yyyy hh24:mi:ss'), 4)
                 

                 
            
        """
execute_query(query)

In [123]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (3, to_timestamp('04-08-2019 23:38:00', 'dd-mm-yyyy hh24:mi:ss'), 4)
            
        """

execute_query(query)

In [124]:
#запрос для выведения id пользователя, имени, полученных лайков, поставленных лайков, взаимных лайков.

query = '''
           select u.user_id,
                  u.name, 
                  count(l.user_id) as put_likes,
                  count(l.user_id_give) as given_likes 
            from users u
            left join likes l on u.user_id = l.user_id
            group by u.user_id, u.name
            
            '''
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Katya', put_likes=3, given_likes=3)
1 Record(user_id=2, name='Julia', put_likes=2, given_likes=2)
2 Record(user_id=3, name='Olya', put_likes=2, given_likes=2)
3 Record(user_id=4, name='Lena', put_likes=1, given_likes=1)
4 Record(user_id=5, name='Dima', put_likes=1, given_likes=1)


In [125]:
query = '''
           select u.user_id,
                  u.name, 
                  count(l.user_id_give) as mutual_likes 
            from users u
            left join likes l on u.user_id = l.user_id
            where u.user_id = l.user_id_give
            group by u.user_id, u.name
            
            '''
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=3, name='Olya', mutual_likes=1)


In [126]:
#Запрос для выведения 5 самых популярных пользователей

query = '''
           select u.user_id,
                  u.name, 
                  count(l.user_id_give) as given_likes
            from users u
            left join likes l on u.user_id = l.user_id_give
            group by u.user_id, u.name
            order by given_likes desc
            limit 5;
            
            '''
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=4, name='Lena', given_likes=4)
1 Record(user_id=3, name='Olya', given_likes=2)
2 Record(user_id=1, name='Katya', given_likes=1)
3 Record(user_id=2, name='Julia', given_likes=1)
4 Record(user_id=5, name='Dima', given_likes=1)


In [127]:
#второе задание. В воображаемой социальной сети есть Пользователи (id, имя), Фото (id, название, автор) и Комментарии К Фото (id, текст, автор, к какому Фото относится). Необходимо добавить возможность для Пользователей ставить лайки другим Пользователям, Фото или Комментариям к Фото.

import psycopg2 as pg_driver

db = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='120303', 
                        host='localhost', 
                        port='5432'
                    )



def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            print(sql_command)
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                "DROP TABLE IF EXISTS photos;",
                "DROP TABLE IF EXISTS comments;",
                "DROP TABLE IF EXISTS types",
                
              
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name       TEXT       NOT NULL, 
                         type_id       TEXT NOT NULL
                    );
                """
                """CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         title    TEXT NOT NULL,
                         user_id    INT       NOT NULL,
                         type_id TEXT NOT NULL
                  );
                """
               
                """CREATE TABLE comments (
                         comment_id INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         comment    TEXT NOT NULL,
                         photo_id INT NOT NULL,
                         type_id INT NOT NULL
                  );
                """
                
                """CREATE TABLE types (
                         type_id    INT       NOT NULL,
                         type    TEXT NOT NULL
                  );
                """
               
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         type_id    TEXT NOT NULL,
                         to_user_id INT NOT NULL,
                         to_photo_id INT NOT NULL,
                         comment_id INT NOT NULL
                  );
                """]

                

execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
DROP TABLE IF EXISTS photos;
DROP TABLE IF EXISTS comments;
DROP TABLE IF EXISTS types
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name       TEXT       NOT NULL, 
                         type_id       TEXT NOT NULL
                    );
                CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         title    TEXT NOT NULL,
                         user_id    INT       NOT NULL,
                         type_id TEXT NOT NULL
                  );
                CREATE TABLE comments (
                         comment_id INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         comment    TEXT NOT NULL,
                         photo_id INT NOT NULL,
                         type_id INT NOT NULL
                  );
                CREATE TABLE types (
                       

In [128]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                                database="postgres", 
                                user='postgres',
                                password='120303', 
                                host='localhost', 
                                port='5432'
                     )
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [129]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='120303', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [130]:
query = """ INSERT INTO users (user_id, name, type_id) 
            VALUES 
                 (1, 'Vadim', 101),
                 (2, 'Dasha', 101),
                 (3, 'Galina', 101),
                 (4, 'Sonya', 101),
                 (5, 'Kirill', 101), 
                 (6, 'Sasha', 101), 
                 (7, 'Vova', 101)
                 
            
        """


query = """ INSERT INTO photos (photo_id, title, user_id, type_id) 
            VALUES 
                 (21, 'IMG_5832.jpg', 1, 202),
                 (22, 'IMG_3219.jpg', 2, 202),
                 (23, 'IMG_0064.jpg', 3, 202),
                 (24, 'IMG_3185.jpg', 4, 202),
                 (25, 'IMG_1443.jpg', 5, 202)
                 
            
        """



query = """ INSERT INTO comments (comment_id, user_id, comment, photo_id, type_id) 
            VALUES 
                 (71, 7, 'Pretty', 8, 303),
                 (72, 1, 'What the place', 9, 303),
                 (73, 4, 'Queen', 10, 303),
                 (74, 2, 'Love it', 11, 303),
                 (75, 6, 'My sunshine', 12, 303)
                 
            
        """


query = """ INSERT INTO types (type_id, type) 
            VALUES 
                 (101, 'user'),
                 (202, 'photo'),
                 (303, 'comment')
                 
            
        """


query = """ INSERT INTO likes (user_id, type_id, to_user_id, to_photo_id, comment_id) 
            VALUES 
                 (1, 202, 5, 21, 71),
                 (2, 101, 2, 25, 75),
                 (3, 303, 1, 24, 73),
                 (4, 101, 7, 21, 71),
                 (5, 202, 6, 21, 71)
                 
        """
execute_query(query)